In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


In [3]:

def gen_data(y_correlations, nr_samples = 1000):

    y_chance = 0.5

    y = (np.random.uniform(low=0, high=1, size=(nr_samples)) < y_chance)

    x = np.empty((nr_samples, len(y_correlations)), dtype='bool')

    for i in range(nr_samples):
        for j in range(len(y_correlations)):
            if y[i]:
                x[i, j] = (np.random.uniform(low=0, high=1) < y_correlations[j])
            else:
                x[i, j] = (np.random.uniform(low=0, high=1) > y_correlations[j])

    # print(np.var(y))
    # print(np.var(x[0, :]))

    return x, y

x, y = gen_data([0.5, 0.9])

In [15]:
def purity_func(p):
    assert 0 <= p <= 1, p
    return 4 * ((0.5-p)**2)

def get_info(left_indices, y):
    right_indices = ~left_indices

    nr_samples_left = sum(left_indices)
    nr_samples_right = sum(right_indices)

    left_purity = purity_func( sum(y[left_indices] == 0) / nr_samples_left ) if nr_samples_left > 0 else 0
    right_purity = purity_func( sum(y[right_indices] == 0) / nr_samples_right ) if nr_samples_right > 0 else 0

    return left_purity * nr_samples_left + right_purity * nr_samples_right



def find_max_gain_feature(x, y, used_features):

    nr_features = x.shape[1]
    nr_samples = x.shape[0]

    maxx = -9999
    threshold = None

    for feature in range(nr_features):
        if feature in used_features:
            continue

        feature_is_categorical = (x.dtypes[feature] == 'bool')

        # print('feature: ', feature, 'is gategorical:', feature_is_categorical)

        if feature_is_categorical:
            feature_info = get_info(x.iloc[:, feature] == 0, y)

        else:
            feature_info = -9999999

            for i in range(nr_samples):
                info = get_info(x.iloc[:, feature] <= x.iloc[i, feature], y) # optimalnije?

                if info > feature_info:
                    feature_info = info
                    threshold = x.iloc[i, feature]

        if feature_info > maxx:
            maxx = feature_info
            max_feature = feature
            max_threshold = threshold
        

    return max_feature, max_threshold

class Node:
    def __init__(self):
        self.left = None
        self.right = None
        self.best_feature = None

def calc_output(y):
    unique, counts = np.unique(y, return_counts=True)

    if len(counts) >= 2:
        if counts[0] > counts[1]:
            return unique[0]
        else:
            return unique[1]
    else:
        return unique[0]

def build(x, y, used_features):

    nr_features = x.shape[1]
    nr_samples = x.shape[0]
    cur = Node()

    cur.info = purity_func((y==0).sum() / len(y))

    if cur.info > 0.999 or nr_features - len(used_features) == 1: # go inside only if tree already predicts well or if this is the last feature

        s = set(np.arange(0, nr_features))
        for i in used_features:
            s.remove(i)

        cur.best_feature = next(iter(s))
        # print('detected last: ', cur.best_feature)
        cur.leaf_output = calc_output(y)
        return cur
    else:
        cur.leaf_output = None
        pass


    cur.best_feature, cur.threshold = find_max_gain_feature(x, y, used_features)

    if cur.threshold is None:
        left_indices = (x.iloc[:, cur.best_feature] == 0)
    else:
        left_indices = (x.iloc[:, cur.best_feature] <= cur.threshold)

    right_indices = ~left_indices

    left = x[left_indices]
    right = x[right_indices]

    new_used_features = list(used_features)
    new_used_features.append(cur.best_feature)

    cur.left = build(left, y[left_indices], new_used_features)
    cur.right = build(right, y[right_indices], new_used_features)

    return cur


def train(x, y):
    x = pd.DataFrame(x)
    # y = pd.DataFrame(y)

    return build(x, y, [])

def predict_helper(node, x):
    if node.leaf_output != None: # this means that "node" is the leaf output node
        return node.leaf_output
    else:
        return predict_helper(node.left, x) if x[node.best_feature] == 0 else predict_helper(node.right, x)

    
def predict(tree, x):
    y = np.empty(len(x), dtype='bool')
    x = pd.DataFrame(x)

    for i in range(len(x)):
        y[i] = predict_helper(tree, x.iloc[i])
    
    return y

def log(node):
    if not node:
        return '#'
    return [log(node.left), node.best_feature, node.info, log(node.right)]

x, y = gen_data([0.5, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6, 0.6])
tree = train(x, y)
y_hat = predict(tree, x)
print('accuracy:', sum(y_hat == y) / len(y) * 100)

a, b = np.unique(y_hat, return_counts=True)
print(a, b)

# log(tree)

accuracy: 74.7
[False  True] [459 541]


In [16]:
df = pd.read_csv('titanic/train.csv')

# TODO: Pclass -> kategorije. Embarked -> kategorije
# TODO: pogledati sta je od preprocessing radio ken jee, mozda nesto da se impute-uje

x = pd.DataFrame(df[['Sex', 'Age', 'SibSp', 'Parch', 'Fare']])
x['Sex'] = (x['Sex'] == 'male')
y = df['Survived']

tree = train(x, y)

y_hat = predict(tree, x)
print('accuracy:', sum(y_hat == y) / len(y) * 100)

a, b = np.unique(y_hat, return_counts=True)
print(a, b)


accuracy: 72.8395061728395
[False  True] [717 174]


In [23]:
test_df = pd.read_csv('titanic/test.csv')

x = pd.DataFrame(test_df[['Sex', 'Age', 'SibSp', 'Parch', 'Fare']])
x['Sex'] = (x['Sex'] == 'male')

y_hat = predict(tree, x)
output = pd.DataFrame({'PassengerId':[], 'Survived':[]})
output.iloc[:, 1] = y_hat.astype('int')
output.iloc[:, 0] = test_df.iloc[:, 0]

output.to_csv('izlaz.csv', index=False)

C:\Users\milos\AppData\Local\Temp\ipykernel_28516\1850523993.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  output.iloc[:, 0] = test_df.iloc[:, 0]
